In [1]:
import pandas as pd
import numpy as np
# from pyabsa.functional import ABSADatasetList
# from pyabsa.functional import ATEPCCheckpointManager
import pickle

In [2]:
# import random
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hdbscan
import umap
import tensorflow as tf

import tensorflow_hub as hub

from sentence_transformers import SentenceTransformer

In [3]:
try:
    with open('POC_sentence_transformer.pkl', 'rb') as file:
        sentence_transformer = pickle.load(file)
except:
    print("failed")


In [4]:
try:
    with open('POC_umap_reducer.pkl', 'rb') as file:
        umap_reducer = pickle.load(file)
except:
    print("failed")


In [5]:
try:
    with open('POC_hdbscan_clusters.pkl', 'rb') as file:
        hdbscan_clusters = pickle.load(file)
except:
    print("failed")


In [6]:
try:
    corpus_clusters = pd.read_csv("POC_clusters.csv")
except:
    print("failed")



In [7]:
corpus_clusters

,Unnamed: 0,label_st1,group
0,0,-1,"cranberry,interior,sever,Hiro,styrofoam,regist..."
1,1,0,"food,food,food,food,food,food,food,food,food,f..."
2,2,1,"Service,service,Service,service,Service,servic..."
3,3,2,"selection,food selection,selection,selection,o..."
4,4,3,"quality,food quality,quality,quality of food,q..."
...,...,...,...
63,63,62,"pasta,pasta,noodles,spaghetti,pasta,pastas,pas..."
64,64,63,"Lobster Bisque soup,Egg Flower Soups,Albondiga..."
65,65,64,"red velvet cake,dessert section,chocolate milk..."
66,66,65,"sandwich,English muffin,bread,pork sandwich,sl..."


In [8]:
topic_categories = [
    'service',
    'hygeine', 
    'appetizer', 
    'desserts',
    'drinks/bar', 
    'food',
    'dishes', 
    'atmosphere', 
    'menu', 
    'price', 
    'management',
    'staff'
]

In [9]:
# def embed(model, model_type, sentences):
#     if model_type == 'use':
#         embeddings = model(sentences)
#     elif model_type == 'sentence transformer':
#         embeddings = model.encode(sentences, show_progress_bar=True)
    
#     return embeddings

In [10]:
def embed(model, sentences):
#     if model_type == 'use':
#         embeddings = model(sentences)
#     elif model_type == 'sentence transformer':
    embeddings = model.encode(sentences, show_progress_bar=True)
    
    return embeddings

In [11]:
topic_categories_embeddings_st1 = embed(sentence_transformer, topic_categories)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
clusters_embedding = embed(sentence_transformer, corpus_clusters.group.to_list())

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
from numpy.linalg import norm
def cosine_sim(A,B):
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine

In [14]:
labels = []
for i in range(len(clusters_embedding)):
    labels.append([cosine_sim(phrase, clusters_embedding[i]) for phrase in topic_categories_embeddings_st1])

In [15]:
test = pd.DataFrame(labels, columns =topic_categories)
test['topic'] = test.idxmax(axis=1)
test['topic_strength'] = test.max(axis=1)
test = test.round(2)

In [16]:
test

,service,hygeine,appetizer,desserts,drinks/bar,food,dishes,atmosphere,menu,price,management,staff,topic,topic_strength
0,0.18,0.21,0.39,0.43,0.29,0.34,0.43,0.14,0.48,0.06,0.10,0.16,menu,0.48
1,0.18,0.22,0.27,0.37,0.28,0.55,0.42,0.17,0.37,0.19,0.21,0.15,food,0.55
2,0.60,0.17,0.19,0.26,0.25,0.27,0.25,0.14,0.30,0.12,0.35,0.32,service,0.60
3,0.10,0.15,0.30,0.36,0.11,0.49,0.24,0.20,0.40,0.10,0.26,0.10,food,0.49
4,0.23,0.12,0.23,0.30,0.13,0.42,0.32,0.16,0.42,0.15,0.27,0.17,food,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.10,0.19,0.38,0.36,0.23,0.45,0.45,0.14,0.39,0.03,0.10,0.06,food,0.45
64,0.03,0.13,0.27,0.31,0.18,0.25,0.30,0.03,0.32,0.02,-0.02,-0.02,menu,0.32
65,0.06,0.06,0.32,0.57,0.18,0.27,0.28,0.04,0.33,0.03,0.01,0.08,desserts,0.57
66,0.09,0.18,0.39,0.36,0.17,0.40,0.33,0.15,0.40,0.01,0.08,0.10,food,0.40


In [17]:
label_strength = pd.concat([corpus_clusters, test[['topic','topic_strength']]], axis=1)
label_strength

,Unnamed: 0,label_st1,group,topic,topic_strength
0,0,-1,"cranberry,interior,sever,Hiro,styrofoam,regist...",menu,0.48
1,1,0,"food,food,food,food,food,food,food,food,food,f...",food,0.55
2,2,1,"Service,service,Service,service,Service,servic...",service,0.60
3,3,2,"selection,food selection,selection,selection,o...",food,0.49
4,4,3,"quality,food quality,quality,quality of food,q...",food,0.42
...,...,...,...,...,...
63,63,62,"pasta,pasta,noodles,spaghetti,pasta,pastas,pas...",food,0.45
64,64,63,"Lobster Bisque soup,Egg Flower Soups,Albondiga...",menu,0.32
65,65,64,"red velvet cake,dessert section,chocolate milk...",desserts,0.57
66,66,65,"sandwich,English muffin,bread,pork sandwich,sl...",food,0.40


In [18]:
label_strength_dict = dict(zip(label_strength['label_st1'] ,label_strength['topic_strength']))

In [19]:
label_dict = dict(zip(label_strength['label_st1'] ,label_strength['topic']))
label_dict[-1] = "Unknown"

In [20]:
try:
    with open('aspect_extractor.pkl', 'rb') as file:
        aspect_extractor = pickle.load(file)


except: 
    print("FAILED!!!")
#     aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english')
#     with open('aspect_extractor.pkl', 'wb') as file:
#         pickle.dump(aspect_extractor, file)

This script could only be used to manage NVIDIA GPUs,but no GPU found in your device


/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from .optimizers import Adam, linear_decay
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, Sized, Iterable, Callable


Remote ABSADataset version: 2022.07.22 Local ABSADatasets version: None
Unknown local version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets


In [21]:
user_review=['The cucumber cocktail was very refreshing, the pork in the hangover ramen was a bit hard to chew...',
            'The pistachio icecream was a delight! Service was ok']

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
atepc_result = aspect_extractor.extract_aspect(inference_source=user_review,
                                               save_result=False,
                                               print_result=True,  # print the result
                                               pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                                               )

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:359: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  lcf_cdm_vec = torch.tensor([f.lcf_cdm_vec for f in infer_features], dtype=torch.float32)


Text: The <cucumber cocktail:Positive> was very refreshing , the <pork:Negative> in the hangover <ramen:Negative> was a bit hard to chew . . .
Text: The <pistachio icecream:Positive> was a delight ! <Service:Neutral> was ok


/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:407: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]


In [23]:
for i in range(len(atepc_result)):
    atepc_result[i]['raw_text'] = user_review[i]

In [24]:
sentiment_dict = {'Positive': 1, 'Negative': -1, 'Neutral': 0}

cleanup = []
for i in atepc_result:
    for j in range(len(i['aspect'])):
        cleanup.append([
            i['raw_text'],
            i['aspect'][j],
#             sentiment_dict[
                i['sentiment'][j],
#             ],
            i['confidence'][j]
        ])
cleanup = pd.DataFrame(cleanup)
cleanup.columns = ['text', 'aspect', 'sentiment', 'confidence']
cleanup

,text,aspect,sentiment,confidence
0,"The cucumber cocktail was very refreshing, the...",cucumber cocktail,Positive,0.999702
1,"The cucumber cocktail was very refreshing, the...",pork,Negative,0.997258
2,"The cucumber cocktail was very refreshing, the...",ramen,Negative,0.862358
3,The pistachio icecream was a delight! Service ...,pistachio icecream,Positive,0.999255
4,The pistachio icecream was a delight! Service ...,Service,Neutral,0.974351


In [25]:
umap_reducer

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


UMAP(angular_rp_forest=True, metric='cosine', n_components=17, n_neighbors=8, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [26]:
test_new_points = cleanup['aspect'].tolist()
test_new_points_embeddings_st1 = embed(sentence_transformer, test_new_points)
test_new_points_embeddings_st1_umap = umap_reducer.transform(test_new_points_embeddings_st1)
test_labels, strengths = hdbscan.approximate_predict(hdbscan_clusters, test_new_points_embeddings_st1_umap)

cleanup['topic'] = [label_dict[i] for i in test_labels]
cleanup['topic_strength'] = [label_strength_dict[i] for i in test_labels]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
# cleanup.raw_text = cleanup.raw_text.astype(str)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
cleanup

,text,aspect,sentiment,confidence,topic,topic_strength
0,"The cucumber cocktail was very refreshing, the...",cucumber cocktail,Positive,0.999702,Unknown,0.48
1,"The cucumber cocktail was very refreshing, the...",pork,Negative,0.997258,menu,0.45
2,"The cucumber cocktail was very refreshing, the...",ramen,Negative,0.862358,Unknown,0.48
3,The pistachio icecream was a delight! Service ...,pistachio icecream,Positive,0.999255,Unknown,0.48
4,The pistachio icecream was a delight! Service ...,Service,Neutral,0.974351,Unknown,0.48


In [29]:
cleanup.confidence = cleanup.confidence.round(2)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
df_display = cleanup.groupby(['text','sentiment']).agg(
    aspect=('aspect', lambda x: list(x)),
    aspect_topic=('topic', lambda x: list(x)),
    sentiment_confidence=('confidence', lambda x: list(x)),
    topic_confidence=('topic_strength', lambda x: list(x)),
)

df_display

aspect  \
text                                               sentiment                         
The cucumber cocktail was very refreshing, the ... Negative          [pork, ramen]   
                                                   Positive    [cucumber cocktail]   
The pistachio icecream was a delight! Service w... Neutral               [Service]   
                                                   Positive   [pistachio icecream]   

                                                                 aspect_topic  \
text                                               sentiment                    
The cucumber cocktail was very refreshing, the ... Negative   [menu, Unknown]   
                                                   Positive         [Unknown]   
The pistachio icecream was a delight! Service w... Neutral          [Unknown]   
                                                   Positive         [Unknown]   

                                                             sentiment_confidence  \
text                                               sentiment                        
The cucumber cocktail was very refreshing, the ... Negative           [1.0, 0.86]   
                                                   Positive                 [1.0]   
The pistachio icecream was a delight! Service w... Neutral                 [0.97]   
                                                   Positive                 [1.0]   

                                                             topic_confidence  
text                                               sentiment                   
The cucumber cocktail was very refreshing, the ... Negative      [0.45, 0.48]  
                                                   Positive            [0.48]  
The pistachio icecream was a delight! Service w... Neutral             [0.48]  
                                                   Positive            [0.48]